In [117]:
import os

In [118]:
%pwd

'C:\\Users\\Mariam\\Desktop\\DataScience\\NLPtutorials\\nlpprojects\\Nlp_text_summarization'

In [119]:
# os.chdir('../')
os.chdir('C:/Users/Mariam/Desktop/DataScience/NLPtutorials/nlpprojects/Nlp_text_summarization/')

In [120]:
%pwd

'C:\\Users\\Mariam\\Desktop\\DataScience\\NLPtutorials\\nlpprojects\\Nlp_text_summarization'

In [121]:
import os
print(os.path.abspath('config\\config.yaml'))

C:\Users\Mariam\Desktop\DataScience\NLPtutorials\nlpprojects\Nlp_text_summarization\config\config.yaml


### Update Entity configuration

In [122]:
from dataclasses import dataclass
from pathlib import Path

# @decorator

@dataclass(frozen=True)
class DataIngestionConfig:
    #return variable types taken from config yaml file
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### Update configuration manager in src

In [123]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [124]:
class ConfigurationManager:
    # access to constants variables
    def __init__(
        self, 
        config_file_path= CONFIG_FILE_PATH, 
        params_filepath= PARAMS_FILE_PATH,
        ):
        self.config= read_yaml(config_file_path) #acces yaml file and read the content
        self.params= read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root]) #configBox method, create the directory created on yaml: artifacts_root
        
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config= self.config.data_ingestion
        
        create_directories([config.root_dir]) #data ingestion folder
        
        #return types
        data_ingestion_config= DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir,
        )
        return data_ingestion_config

### Components

In [125]:
import os
import urllib.request as request # to download data from url
import zipfile # to perform unzip operation
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [126]:
# data Ingestion component , it will take configurations: dataingestionconfiguration
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config= config
    
    #downlaod data.zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

### Create Pipeline

In [127]:
try:
    config=ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2024-01-10 23:05:38,448: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-10 23:05:38,453: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-10 23:05:38,456: INFO: common: created directory at: artifacts]
[2024-01-10 23:05:38,458: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-10 23:05:42,617: INFO: 2178320469: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5B36:388ABE:1CDE1C6:1E02819:659EF891
Accept-Ranges: bytes
Date: Wed, 10 Jan 2024 20:05:39 GMT
Via: 1.1 varnish
X-Served-By: cache-vie6345-VIE
X-Cache: 

In [128]:
# print(os.getcwd())